# Search 程序



In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import random 

import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams["font.family"] = 'Arial Unicode MS'
# mpl.rcParams["font.sans-serif"] = ["Arial Unicode MS"]
mpl.rcParams["axes.unicode_minus"] = False

# import mplfinance as mpf

from importlib import reload 
from qflib import basic,search
reload(basic)
reload(search)

# data_dir='../'
data_dir='~/qfdata/'
engine = basic.engine()


In [2]:
# 初始化 bt_list
''' 测试清单 '''
bt_list = '../data/stocklist.csv'
bt_list = pd.read_csv(bt_list)
start_date = '2019-01-01'
end_date= '' # '2022-01-20'


viewlist = pd.DataFrame(columns=['ts_code'])
viewlist.to_csv('../viewlist.csv', header=True,index=False)

# 初始化bt_result, 无记录
# bt_result = pd.DataFrame(columns=['ts_code', 'start_date', 'end_date','pct_profit','pct_noaction','pct_diff'])
# print(bt_result)


In [3]:
%%time
reload(search)
for i in range(0, len(bt_list) ):
    
    ts_code = bt_list.loc[i,'ts_code']
    ts_cal_file = data_dir+'cal/' + ts_code+'.cal.csv'    
    print( '', i, ts_code, end="")
    # print( '------------------')
    df = pd.read_csv(ts_cal_file, index_col='trade_date', parse_dates=True)
    # df = basic.read_data(engine, "select * from stock_cal where ts_code='"+ bt_list.loc[i,'ts_code'] + "'")
    
    # 计算买卖点
    df['bt_s1'] = 0  # View显示字段, 1类买卖点  
    df['bt_dtts'], df['bt_xrds'], df['bt_qmzx'], df['bt_chuizi'] = 0, 0, 0, 0  # 多头吞噬, 旭日东升, 启明之星，锤子Plus 
    df['bt_zonghe']  = 0  # charlie: 综合 
    df['bt_stp'] = 0  # 双头炮 
    df['bt_pingtai'] = 0  # 平台模式 

    # df['pct_st_15d_max']=df['pct_st_range'].rolling(15).max()
    buy_day, sale_day = [], []
    run_xrds, run_dtts, run_qmzx, run_chuizi = False, False, False, False
    run_zonghe = False
    run_stp = True
    run_pingtai = True
    # change_zhongyang = 5    # 大阳涨幅

    # # 旭日东升模式 xrdf 
    # # 連續3天下跌、今天高开、高收、中阳以上
    # if run_xrds == True:
    #     res = backtest.xrds(df, 5)  
    #     # def : 今日涨幅 
    #     buy_day += res[0]

    # # 多头吞噬法买， 空头吞噬法卖出
    # # 低开高走：今天实体低点低于昨天低点， 高点高于昨天高点； 中阳以上
    # if run_dtts == True:
    #     res = backtest.dtts(df, 5)  
    #     # dtts(df, pct_change_td-今日涨幅)
    #     buy_day += res[0]
        
    # # 启明之星模式 qmzx 
    # # 启明之星 = 前日阴线 + 实体缺口 + 星线 + 今日的多头吞噬(低开高走) 或 旭日东升(高开高走)
    # if run_qmzx == True:
    #     res = backtest.qmzx(df, 3)
    #     buy_day += res[0]

    # # 锤子 + 倒锤子
    # if run_chuizi == True: 
    #     res = backtest.chuizi(df, -1.5, 2, 2, 2)   # 昨日长下引线+今日中阳+短上引线
    #     # df, 前3日平均跌幅，昨日下引线，今日涨幅, 今日上引线
    #     # def chuizi(df, pct_fall, pct_down_ys, pct_change_td, pct_upper_td):
    #     # df, 前3日平均跌幅-1.5，昨日下引线2，今日涨幅2, 今日上引线2; 
    #     # ys: yesterday; td: today
    #     buy_day += res[0]

    # # 综合 - Charlie
    # if run_zonghe == True:
    #     # res = backtest.czkz(df, -1.5, 2, 4, 8)    # 连续下跌后反弹明显
    #     res = backtest.zonghe(df, -0.5, 4, 2, 6)      # 昨日长下引线
    #     # df, 前3日平均跌幅，昨日下引线，今日涨幅, 昨日今日2日振幅
    #     buy_day += res[0]

    # 双头跑
    # if run_stp == True:
    #     res = backtest.stp(df, 5, 0.1, 6)    # 昨日长下引线
    #     # def stp(df, change_1st, daily_change, change_td):
    #     # df, 首日涨幅8%， 每天平均波动0.75，今日涨幅
    #     buy_day += res[1]
    #     df = res[0]

    if run_pingtai == True:
        res = search.pingtai(df)    # 昨日长下引线
        # def stp(df, change_1st, daily_change, change_td):
        # df, 首日涨幅8%， 每天平均波动0.75，今日涨幅
        if res[1]:
            newone = pd.DataFrame( [[ bt_list.loc[i,'ts_code'] ]], columns=['ts_code'] )
            newone.to_csv('../data/viewlist.csv', mode='a', header=False, index=False)
            viewlist = pd.concat( [viewlist, newone] )
        df = res[0]
        
    df['bt_s1']  = df['bt_pingtai']
    # df['bt_s1']  = df.apply(search.merge_actions, axis=1)   # 实体部分百分比

    # print( 'Buy date : ', buy_day )
    # print( 'Sale date：', sale_day )

    df.to_csv( data_dir+'bt/' + ts_code + '.bt.csv' )
    
    ''' end 0.5 '''


print('viewlist：')
print(viewlist.shape[0])
print(viewlist)

# df[ df['pingtai']==1 ]

# print( bt_result )
# bt_result.to_csv( '../bt/bt_result.csv' )


 0 002910.SZ 1 000016.SZ 2 001872.SZ 3 000040.SZ 4 000401.SZ 5 000429.SZ 6 000553.SZ 7 000560.SZ 8 000710.SZ 9 000723.SZ 10 000951.SZ 11 000968.SZ 12 002061.SZ 13 002507.SZ 14 002530.SZ 15 002537.SZ 16 002545.SZ 17 002611.SZ 18 002648.SZ 19 002747.SZ 20 002812.SZ 21 002839.SZ 22 002841.SZ 23 002859.SZ 24 002867.SZ 25 300308.SZ 26 300457.SZ 27 300616.SZ 28 300618.SZ 29 300628.SZ 30 300666.SZ 31 002936.SZ 32 000301.SZ 33 000537.SZ 34 000546.SZ 35 000789.SZ 36 000831.SZ 37 000932.SZ 38 000935.SZ 39 001965.SZ 40 002258.SZ 41 002382.SZ 42 002409.SZ 43 002430.SZ 44 002511.SZ 45 002677.SZ 46 002807.SZ 47 002901.SZ 48 002907.SZ 49 002911.SZ 50 002912.SZ 51 002913.SZ 52 002916.SZ 53 002920.SZ 54 300232.SZ 55 300298.SZ 56 300349.SZ 57 300357.SZ 58 300482.SZ 59 300529.SZ 60 300567.SZ 61 300601.SZ 62 300676.SZ 63 300679.SZ 64 300685.SZ 65 300699.SZ 66 300725.SZ 67 300726.SZ 68 300735.SZ 69 000025.SZ 70 000617.SZ 71 000717.SZ 72 000818.SZ 73 000877.SZ 74 000933.SZ 75 000976.SZ 76 000980.SZ 77 00211

In [4]:
# type( bt_result.loc[0] )

In [11]:

df0 = pd.DataFrame(data=[1,2,3])
df0


,0
0,1
1,2
2,3
